In [1]:
import shutup
shutup.please()

import gym
import src.environments.continuous.stock_trading  

import numpy as np
import pandas as pd 

In [2]:
env = gym.make('StockTradingEnvironment-v0', use_technical_indicators= [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
])

In [3]:
pd.read_csv('src/environments/continuous/stock_trading/datasets/67b906a1ca1d5e7e03dd20f0c89fd03c/targets.csv').targets.iloc[0]

7.170001983642585

In [4]:
env.train_dataframe_id_range,env.test_dataframe_id_range

([0, 315], [316, 395])

In [5]:
env.load_dataset_by_index(0)
env.df

,open,high,low,close,volume,day,vix,turbulence,macd,boll_ub,...,neg_title,neu_title,pos_title,compound_title,neg_body,neu_body,pos_body,compound_body,aapl,fear greed
date,,,,,,,,,,,,,,,,,,,,,
2018-01-08,43.587502,43.902500,43.482498,43.587502,82271200,0,0.0952,0.061110,-0.003477,41.870444,...,0.000000,1.000000,0.000000,0.000000,0.008000,0.934000,0.058000,0.963700,0.395833,0.75
2018-01-09,43.637501,43.764999,43.352501,43.582500,86336000,1,0.1008,0.065026,-0.004579,41.805620,...,0.000000,0.946800,0.053200,0.088080,0.026400,0.919000,0.055000,0.684980,0.397083,0.75
2018-01-11,43.647499,43.872501,43.622501,43.820000,74670800,3,0.0988,0.060938,0.003795,41.864027,...,0.244000,0.756000,0.000000,-0.296525,0.041000,0.863750,0.095250,0.636025,0.379167,0.77
2018-01-12,44.044998,44.340000,43.912498,44.272499,101672400,4,0.1016,0.070963,0.027654,42.256032,...,0.000000,1.000000,0.000000,0.000000,0.126000,0.874000,0.000000,-0.893400,0.371667,0.79
2018-01-16,44.474998,44.847500,44.035000,44.047501,118263600,1,0.1166,0.172370,0.032756,42.292433,...,0.000000,0.931667,0.068333,0.067433,0.030000,0.849333,0.120667,0.729800,0.378333,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-04-30,40.532501,41.814999,40.459999,41.314999,169709600,0,0.1593,0.217870,-0.531165,43.128501,...,0.000000,1.000000,0.000000,0.000000,0.022500,0.877500,0.100500,0.939450,0.418333,0.42
2018-05-01,41.602501,42.299999,41.317501,42.275002,214277600,1,0.1549,0.087329,-0.454444,43.132534,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.962000,0.038000,0.381800,0.450417,0.38
2018-05-02,43.807499,44.437500,43.450001,44.142502,266157600,2,0.1597,0.128597,-0.246744,43.287283,...,0.040364,0.826455,0.133182,0.110127,0.063545,0.853364,0.083091,0.339791,0.431250,0.39


In [6]:
env.success_threshold

0.030329969111030485

In [7]:
import pandas as pd
episode_targets = []
for i in range(env.n_dataframes):
    env.load_dataset_by_index(i)
    raw_slice = env.df 
    episode_targets.append([
                    env.get_episode_target(raw_slice),
                    raw_slice.iloc[env.lookback-1].open,
                    raw_slice.iloc[env.lookback-1].low,
                    raw_slice.iloc[env.lookback-1].high,
                    raw_slice.iloc[env.lookback-1].close,
                    raw_slice.iloc[env.lookback-1].volume,
                ])




pd.DataFrame(episode_targets,columns=['targets','open','low','high','close','volume']).to_csv(env.df_path+'/targets.csv', index=False)

In [8]:
episode_columns = ['targets','close','open','high','low','volume']

#pd.DataFrame([[1,2],[1,2]],columns=['a','b'])
#np.array(episode_targets).shape ,=['targets','close','open','high','low','volume']).
pd.DataFrame(episode_targets,columns=episode_columns)

,targets,close,open,high,low,volume
0,7.170002,44.325001,44.205002,44.860001,44.259998,130756400.0
1,7.502502,44.312500,43.299999,44.325001,43.555000,204420400.0
2,7.725002,43.627499,42.632500,43.737499,42.777500,166116000.0
3,8.052502,42.540001,41.767502,42.540001,41.990002,202561600.0
4,8.722500,41.382500,41.174999,41.842499,41.742500,184192800.0
...,...,...,...,...,...,...
390,19.575001,54.432499,54.389999,55.032501,54.974998,84632400.0
391,19.642498,54.990002,54.779999,55.205002,55.174999,73274800.0
392,20.180000,55.264999,54.860001,55.712502,55.692501,101360000.0
393,19.157501,55.502499,55.092499,55.939999,55.240002,88242400.0


In [9]:
state = env.reset()

In [10]:
state

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.86512081e-02, -1.13471649e-02,
         1.07726200e-02,  1.04000419e-02, -4.18027950e-01,
         0.00000000e+00,  1.87299995e-01,  4.28617255e-02,
         5.26999824e+03,  4.26898976e+03,  3.51753024e+03,
         5.54725913e-01,  3.02123771e-01,  7.49638856e-03,
         3.89591012e+03,  3.93267591e+03,  0.00000000e+00,
         7.41000000e-01,  2.59000000e-01,  5.42300000e-01,
         4.50000000e-02,  8.15000000e-01,  1.40000000e-01,
         9.69800000e-01,  3.90833333e-01,  1.40000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -4.01651926e-03, -4.65062762e-03,
        -3.02723914e-02, -1.75162542e-02,  2.91669865e-01,
         7.50000000e-01,  2.24699993e-01,  1.72600837e-01,
         1.78209365e+03, -2.51204023e+03, -2.07545018e+

In [11]:
next_state = env.step([1])
next_state 

(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -4.01651926e-03, -4.65062762e-03,
         -3.02723914e-02, -1.75162542e-02,  2.91669865e-01,
          7.50000000e-01,  2.24699993e-01,  1.72600837e-01,
          1.78209365e+03, -2.51204023e+03, -2.07545018e+03,
          5.37356834e-01,  2.62733572e-01,  3.64370682e-02,
         -2.31461963e+03, -2.33088246e+03,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  3.92916667e-01,  8.00000000e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  8.93290887e-04,  7.75705856e-04,
         -6.17425955e-04, -1.51356171e-03, -2.94388886e-02,
          5.00000000e-01,  1.98500004e-01,  7.33090460e-02,
          2.41698169e+05, -2.89493778e+

In [12]:
next_state[0][0] == state[1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [13]:
# env.reset()
# for i in range(100):
#     next_state, reward, done, _ = env.step(0)
#     print(i, reward,done)#,next_state[-1])
#     if done:
#         break

In [14]:
len(env.df),env.window_size, env.lookback, env.window_size+ env.lookback

(76, 66, 10, 76)

In [15]:
# prices = [5,2,1,7,0,6,3]

# """
# # mins = min de inicio até i inclusive 
#     0 - 5
#     1 - 2
#     2 - 2
#     3 - 2
# # maxs = max de i até fim inclusive
#     0 - 6
#     1 - 6
#     2 - 6
#     3 - 3
# """

# mins = [] #[5,2,2,2]
# maxs = [] #[6,6,6,3]

# ### MINS
# # i = 0 -> prices [0 -> 1] [0:1]
# # mins.append(min(prices[0:1]))
# ### MAXS
# # i = 0 -> prices[0:]
# # i = 1 -> prices[1:]
# # prices[3:]

# for i in range(len(prices)):
#     mins.append(min(prices[0:i+1]))
#     maxs.append(max(prices[i:]))
    
# #mins,maxs
# global_max_single_trade_profit = 0
# for i in range(len(prices)):
#     _min = mins[i]
#     _max = maxs[i]
    
#     print('At {} could have bought at {} and sold at {} so it had {} profit'.format(
#         i,
#         _min,
#         _max,
#         _max-_min
#     ))
#     local_max_single_trade_profit = _max-_min
#     if local_max_single_trade_profit > global_max_single_trade_profit:
#         global_max_single_trade_profit = local_max_single_trade_profit

# print()
# print('Best case scenario would have a profit of {}'.format(global_max_single_trade_profit))

In [16]:
# for i in range(len(prices)):
#     mins.append(min(prices[0:i+1]))
#     maxs.append(max(prices[i:]))
    
# #mins,maxs
# global_max_single_trade_profit = 0
# for i in range(len(prices)):
#     _min = mins[i]
#     _max = maxs[i]
    
#     print('At {} could have bought at {} and sold at {} so it had {} profit'.format(
#         i,
#         _min,
#         _max,
#         _max-_min
#     ))
#     local_max_single_trade_profit = _max-_min
#     if local_max_single_trade_profit > global_max_single_trade_profit:
#         global_max_single_trade_profit = local_max_single_trade_profit

# print()
# print('Best case scenario would have a profit of {}'.format(global_max_single_trade_profit))

In [17]:
# prices = env.df.close.values

# mins = [] 
# maxs = [] 

# for i in range(len(prices)):
#     mins.append(min(prices[0:i+1]))
#     maxs.append(max(prices[i:]))
# #     
# #mins,maxs
# global_max_single_trade_profit = 0
# for i in range(len(prices)):
#     _min = mins[i]
#     _max = maxs[i]
    
#     print('At {} could have bought at {} and sold at {} so it had {} profit'.format(
#         i,
#         _min,
#         _max,
#         _max-_min
#     ))
#     local_max_single_trade_profit = _max-_min
#     if local_max_single_trade_profit > global_max_single_trade_profit:
#         global_max_single_trade_profit = local_max_single_trade_profit

# print()
# print('Best case scenario would have a profit of {}'.format(global_max_single_trade_profit))

In [18]:
#prices * (0.5)
#env.fees['BUY']

In [19]:
# env.n_dataframes

In [20]:
# targets = []
# for i in range(env.n_dataframes):
#     env.load_dataset_by_index(i)
#     target = env.calculate_episode_target(env.df)
#     targets.append(target)
    
# targets

In [21]:
# import pandas as pd
# pd.DataFrame(targets,columns=['targets']).to_csv(env.df_path+'/targets.csv', index=False)

In [22]:
#pd.read_csv(env.df_path+'/targets.csv')

In [23]:
from src.agents.actor_critic.a2c import A2CAgent

In [24]:
def environment():
    env = gym.make('StockTradingEnvironment-v0',
                   use_technical_indicators= [
        "macd",
        "boll_ub",
        "boll_lb",
        "rsi_30",
        "cci_30",
        "dx_30",
        "close_30_sma",
        "close_60_sma",
    ])
    
    env.success_threshold =0.05 # 5%
    return env


In [25]:
agent=A2CAgent(environment, epochs=1, actor_learning_rate=0.000025,critic_learning_rate=0.000025)
agent.load()
environment().success_threshold

0.05

In [26]:
agent.learning_log.episodes

0

In [ ]:
agent.env.mode = "train"
agent.learning_log.episodes = 0
agent.learn(
    timesteps=-1, 
    log_every=1000,
    success_threshold_lookback=1000,
    success_strict=True,
    
)

In [ ]:
agent.env.mode = "test"
scores = []
success = 0

for i in range(100):
    state = agent.env.reset()
    score = 0
    reward = 0
    done = False
    initial_portfolio = agent.env.portfolio_value
    step = 0
    while not done:
                
        #state = np.expand_dims(state, axis=0)
        action, action_onehot, prediction = agent.act(state)
        # Retrieve new state, reward, and whether the state is terminal
        next_state, reward, done, _ = agent.env.step(action)
        #print(action, reward, agent.env.portfolio_value)
        # Memorize (state, action, reward) for training
        #self.buffer.remember(np.expand_dims(state, axis=0), action_onehot, reward)
        # Update current state
        if done :
            if step < 65:
                success +=1
        step+=1
        state = next_state
        score += reward
    
    #print(score,initial_portfolio, agent.env.portfolio_value)
    scores.append(score)
    
print(np.mean(scores))
print(min(scores))
print(max(scores))
print(success)